In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [0]:
catalog_dev = "`land_topografisk-gdb_dev`"
schema_dev = "ai2025"

spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
silver_table = "polygons_silver"
uten_snuplass_table = "utensnuplass_silver"
endepunkt_table = "endepunkt_silver"
hospitals_table = "hospitals_silver"
helipads_table = "helipads_silver"

In [0]:
df_retable = spark.read.table(silver_table)
df_overview = df_retable.drop(
    "Shape_Length",
    "Shape_Area",
    "Vegtyper",
    "Snuplasstype",
    "Parkering",
    "geometry",
    "ingest_time",
    "source_file",
    "source_layer",
    "bbox",
    "Polygons",
    "adjusted_struct",
    "Adjusted_bbox",
    "bbox_str",
    "image_wms",
    "dom_wms",
    "lastet_tid",
    "fototid",
)

df_overview.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("polygons_status_overview")

In [0]:
df_retable = spark.read.table(uten_snuplass_table)
df_overview = df_retable.drop(
    "Shape_Length",
    "Shape_Area",
    "bbox",
    "id",
    "ingest_time",
    "source_file",
    "source_layer",
    "geometry",
    "Polygons",
    "adjusted_struct",
    "Adjusted_bbox",
    "bbox_str",
    "image_wms",
    "dom_wms",
    "lastet_tid",
    "fototid"
)
 
df_overview.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("utenSnuplass_status_overview")

In [0]:
df_retable = spark.read.table(endepunkt_table)
df_overview = df_retable.drop(
    "x",
    "y",
    "wkt",
    "kommune_id",
    "hentet_tid",
    "bbox",
    "bbox_str",
    "image_wms",
    "dom_wms",
    "lastet_tid",
    "fototid"
)

df_overview.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("endepunkt_status_overview")

In [0]:
df_retable = spark.read.table(hospitals_table)
df_overview = df_retable.drop(
    "bygningsnummer",
    "geometry",
    "bbox",
    "bbox_str",
    "image_wms",
    "kommunenummer",
    "oppdateringsdato",
    "ingest_time",
    "load_time",
    "photo_time"
)

df_overview.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("hospitals_status_overview")

In [0]:
df_retable = spark.read.table(helipads_table)
df_overview = df_retable.drop(
    "geometry",
    "bbox",
    "Polygons",
    "adjusted_struct",
    "Adjusted_bbox",
    "bbox_str",
    "image_wms",
    "oppdateringsdato",
    "ingest_time",
    "load_time",
    "photo_time",
)

df_overview.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("helipads_status_overview")

In [0]:

poly_over_tbl = "polygons_status_overview"
us_over_tbl = "utensnuplass_status_overview"

df_yes = spark.read.table(poly_over_tbl)
df_no = spark.read.table(us_over_tbl)

In [0]:
count_yes = df_yes.count()
count_no = df_no.count()
p_no = 0.05
desired_no = int((count_yes / (1 - p_no)) * p_no)

frac = desired_no / float(count_no)
df_no_sampled = df_no.sample(withReplacement=False, fraction=frac).limit(desired_no)
df_train = df_no_sampled.union(df_yes)
df_train.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("train_silver")